## Notebook Objective and Contents:
This notebook is based on individual dataset understanding for train, validation and test on DOCVQA
- This notebook doesn't contain RAW Image path
- This notebook also doesn't contain and index and bbox info part i.e no such Fuzzy applied
- After experiment of the training, validation and test we will create custom function to automate all the above steps

In [ ]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import os
import json
from google.colab import data_table
import datasets
from datasets import Dataset
import json
import re
import random
from PIL import Image

## Exp_1: Little details of training dataset

In [ ]:
train_path_main = '/content/drive/MyDrive/docVqa_dataset/train/train_v1.0.json'
# Open the JSON file
with open(train_path_main, 'r') as file:
  # Load the contents of the file into a dictionary
  data_train = json.load(file)

## Convert 'data' list into dataframe by passing into the dataframe
df_train = pd.DataFrame(data_train['data'])

## Making subset of the training dataframe
df_sub_train = df_train.iloc[:10]

base_train_path = '/content/drive/MyDrive/docVqa_dataset/train'

# Define a function to join the base path with the image path
def join_paths(image_path):
    return os.path.join(base_train_path, image_path)

# Apply the function to create a new column with the full image path
df_sub_train['full_path_image'] = df_sub_train['image'].apply(join_paths)


def extract_ocr_info(df, base_train_dir):

  dir_list = os.listdir(base_train_path)
  ocr_base_dir = os.path.join(base_train_path, dir_list[1])

  # Create empty lists for bounding boxes and word lists
  bbox_list = []
  word_list = []

  # Iterate over each row in the DataFrame
  for _, row in df.iterrows():

    # Construct the ocr_path for the row
    ocr_path = os.path.join(ocr_base_dir, row['ucsf_document_id'] + '_' + row['ucsf_document_page_no'] + '.json')

    # Open the json file and load the data
    with open(ocr_path, 'r') as f:
      ocr_json = json.load(f)

    # Extract the bounding boxes and word lists from the json data
    bbox_row = []
    word_row = []
    for i in ocr_json['recognitionResults'][0]['lines']:
      for j in i['words']:
        bbox_row.append(j['boundingBox'])
        word_row.append(j['text'])

    # Append the bbox and word lists to the global lists
    bbox_list.append(bbox_row)
    word_list.append(word_row)
  # Create new columns in the DataFrame to store the bbox and words lists
  df['bounding_boxes'] = bbox_list
  df['word_list'] = word_list

  df.drop(['questionId', 'image','ucsf_document_id', 'ucsf_document_page_no'], axis=1, inplace=True)

  # Return the updated DataFrame
  return df


df_sub_train_ocr = extract_ocr_info(df_sub_train, base_train_path)

## Convert dataframe into dataset object
hf_sub_train_ocr = Dataset.from_pandas(df_sub_train_ocr)


<ipython-input-3-4447ede6effb>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub_train['full_path_image'] = df_sub_train['image'].apply(join_paths)
<ipython-input-3-4447ede6effb>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bounding_boxes'] = bbox_list
<ipython-input-3-4447ede6effb>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [ ]:
hf_sub_train_ocr

Dataset({
    features: ['question', 'docId', 'answers', 'data_split', 'full_path_image', 'bounding_boxes', 'word_list'],
    num_rows: 10
})

In [ ]:
hf_sub_train_ocr.column_names

['question',
 'docId',
 'answers',
 'data_split',
 'full_path_image',
 'bounding_boxes',
 'word_list']

## Exp_2: Little details of validation dataset separately

In [ ]:
## **
valid_path_main = '/content/drive/MyDrive/docVqa_dataset/val/val_v1.0.json'
# Open the JSON file
with open(valid_path_main, 'r') as file:
  # Load the contents of the file into a dictionary
  data_valid = json.load(file)

In [ ]:
type(data_valid)

dict

In [ ]:
data_valid.keys()

dict_keys(['dataset_name', 'dataset_version', 'dataset_split', 'data'])

In [ ]:
## ** Convert 'data' list into dataframe by passing into the dataframe
df_valid = pd.DataFrame(data_valid['data'])

In [ ]:
df_valid.head()

,questionId,question,image,docId,ucsf_document_id,ucsf_document_page_no,answers,data_split
0,49153,"What is the ‘actual’ value per 1000, during th...",documents/pybv0228_81.png,14465,pybv0228,81,[0.28],val
1,24580,What is name of university?,documents/nkbl0226_1.png,7027,nkbl0226,1,"[university of california, University of Calif...",val
2,57349,What is the name of the company?,documents/snbx0223_22.png,4733,snbx0223,22,"[itc limited, ITC Limited]",val
3,24581,Where is the university located ?,documents/nkbl0226_1.png,7027,nkbl0226,1,"[san diego, San Diego]",val
4,24582,To whom is the document sent?,documents/nkbl0226_1.png,7027,nkbl0226,1,[Paul],val


In [ ]:
df_valid.shape    ## validation dataset contains 5349 number of datapoints

(5349, 8)

In [ ]:
## Making subset of the training dataframe
df_sub_valid = df_valid.iloc[:10]

base_valid_path = '/content/drive/MyDrive/docVqa_dataset/val'

# Define a function to join the base path with the image path
def join_paths(image_path):
    return os.path.join(base_valid_path, image_path)

# Apply the function to create a new column with the full image path
df_sub_valid['full_path_image'] = df_sub_valid['image'].apply(join_paths)

<ipython-input-12-084e9d8626f2>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub_valid['full_path_image'] = df_sub_valid['image'].apply(join_paths)


In [ ]:
df_sub_valid.columns

Index(['questionId', 'question', 'image', 'docId', 'ucsf_document_id',
       'ucsf_document_page_no', 'answers', 'data_split', 'full_path_image'],
      dtype='object')

In [ ]:
df_sub_valid.head()

,questionId,question,image,docId,ucsf_document_id,ucsf_document_page_no,answers,data_split,full_path_image
0,49153,"What is the ‘actual’ value per 1000, during th...",documents/pybv0228_81.png,14465,pybv0228,81,[0.28],val,/content/drive/MyDrive/docVqa_dataset/val/docu...
1,24580,What is name of university?,documents/nkbl0226_1.png,7027,nkbl0226,1,"[university of california, University of Calif...",val,/content/drive/MyDrive/docVqa_dataset/val/docu...
2,57349,What is the name of the company?,documents/snbx0223_22.png,4733,snbx0223,22,"[itc limited, ITC Limited]",val,/content/drive/MyDrive/docVqa_dataset/val/docu...
3,24581,Where is the university located ?,documents/nkbl0226_1.png,7027,nkbl0226,1,"[san diego, San Diego]",val,/content/drive/MyDrive/docVqa_dataset/val/docu...
4,24582,To whom is the document sent?,documents/nkbl0226_1.png,7027,nkbl0226,1,[Paul],val,/content/drive/MyDrive/docVqa_dataset/val/docu...


In [ ]:
def extract_ocr_info(df, base_dir):

  dir_list = os.listdir(base_dir)
  ocr_base_dir = os.path.join(base_dir, dir_list[1])

  # Create empty lists for bounding boxes and word lists
  bbox_list = []
  word_list = []

  # Iterate over each row in the DataFrame
  for _, row in df.iterrows():

    # Construct the ocr_path for the row
    ocr_path = os.path.join(ocr_base_dir, row['ucsf_document_id'] + '_' + row['ucsf_document_page_no'] + '.json')

    # Open the json file and load the data
    with open(ocr_path, 'r') as f:
      ocr_json = json.load(f)

    # Extract the bounding boxes and word lists from the json data
    bbox_row = []
    word_row = []
    for i in ocr_json['recognitionResults'][0]['lines']:
      for j in i['words']:
        bbox_row.append(j['boundingBox'])
        word_row.append(j['text'])

    # Append the bbox and word lists to the global lists
    bbox_list.append(bbox_row)
    word_list.append(word_row)
  # Create new columns in the DataFrame to store the bbox and words lists
  df['bounding_boxes'] = bbox_list
  df['word_list'] = word_list

  df.drop(['questionId', 'image','ucsf_document_id', 'ucsf_document_page_no'], axis=1, inplace=True)

  # Return the updated DataFrame
  return df

In [ ]:
base_valid_path

'/content/drive/MyDrive/docVqa_dataset/val'

In [ ]:
df_sub_valid_ocr = extract_ocr_info(df_sub_valid, base_valid_path)

<ipython-input-15-9bb3011757be>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bounding_boxes'] = bbox_list
<ipython-input-15-9bb3011757be>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['word_list'] = word_list
<ipython-input-15-9bb3011757be>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['question

In [ ]:
df_sub_valid_ocr.columns

Index(['question', 'docId', 'answers', 'data_split', 'full_path_image',
       'bounding_boxes', 'word_list'],
      dtype='object')

In [ ]:
df_sub_valid_ocr.head()

,question,docId,answers,data_split,full_path_image,bounding_boxes,word_list
0,"What is the ‘actual’ value per 1000, during th...",14465,[0.28],val,/content/drive/MyDrive/docVqa_dataset/val/docu...,"[[1083, 226, 1209, 226, 1208, 259, 1082, 261],...","[FIGURE, C., 2., AGE, ADJUSTED, MOTOR, VEHICLE..."
1,What is name of university?,7027,"[university of california, University of Calif...",val,/content/drive/MyDrive/docVqa_dataset/val/docu...,"[[36, 72, 200, 65, 200, 96, 36, 94], [208, 65,...","[UNIVERSITY, OF, CALIFORNIA,, SAN, DIEGO, To, ..."
2,What is the name of the company?,4733,"[itc limited, ITC Limited]",val,/content/drive/MyDrive/docVqa_dataset/val/docu...,"[[127, 133, 190, 134, 190, 169, 126, 169], [20...","[ITC, Limited, REPORT, AND, ACCOUNTS, 2013, IT..."
3,Where is the university located ?,7027,"[san diego, San Diego]",val,/content/drive/MyDrive/docVqa_dataset/val/docu...,"[[36, 72, 200, 65, 200, 96, 36, 94], [208, 65,...","[UNIVERSITY, OF, CALIFORNIA,, SAN, DIEGO, To, ..."
4,To whom is the document sent?,7027,[Paul],val,/content/drive/MyDrive/docVqa_dataset/val/docu...,"[[36, 72, 200, 65, 200, 96, 36, 94], [208, 65,...","[UNIVERSITY, OF, CALIFORNIA,, SAN, DIEGO, To, ..."


In [ ]:
## Convert dataframe into dataset object
hf_sub_valid_ocr = Dataset.from_pandas(df_sub_valid_ocr)

In [ ]:
hf_sub_valid_ocr

Dataset({
    features: ['question', 'docId', 'answers', 'data_split', 'full_path_image', 'bounding_boxes', 'word_list'],
    num_rows: 10
})

In [ ]:
hf_sub_valid_ocr.column_names

['question',
 'docId',
 'answers',
 'data_split',
 'full_path_image',
 'bounding_boxes',
 'word_list']

In [ ]:
hf_sub_train_ocr.column_names == hf_sub_valid_ocr.column_names  ## result is true means we are in a right track

True

## Exp_3: Little details of test dataset separately

In [ ]:
test_path_main = '/content/drive/MyDrive/docVqa_dataset/test/test_v1.0.json'
# Open the JSON file
with open(test_path_main, 'r') as file:
  # Load the contents of the file into a dictionary
  data_test = json.load(file)

In [ ]:
type(data_test)

dict

In [ ]:
data_test.keys()

dict_keys(['dataset_name', 'dataset_version', 'dataset_split', 'data'])

In [ ]:
## ** Convert 'data' list into dataframe by passing into the dataframe
df_test = pd.DataFrame(data_test['data'])

In [ ]:
df_test.head()

,questionId,question,image,docId,ucsf_document_id,ucsf_document_page_no,data_split
0,57344,What is the dividend payout in 2012?,documents/rnbx0223_193.png,4720,rnbx0223,193,test
1,16384,What is the name of the person in the CC field ?,documents/lflm0081_1.png,5160,lflm0081,1,test
2,57346,What is the % of Employees in 2012 based on gr...,documents/rnbx0223_191.png,4753,rnbx0223,191,test
3,61870,What is the personnel costs in the 4th year?,documents/hrfw0227_24.png,8103,hrfw0227,24,test
4,57348,What is the % of 'Providers of Capital' in the...,documents/rnbx0223_191.png,4753,rnbx0223,191,test


In [ ]:
df_test.shape    ## test dataset contains 5188 number of datapoints

(5188, 7)

In [ ]:
## Making subset of the training dataframe
df_sub_test = df_test.iloc[:10]

base_test_path = '/content/drive/MyDrive/docVqa_dataset/test'

# Define a function to join the base path with the image path
def join_paths(image_path):
    return os.path.join(base_test_path, image_path)

# Apply the function to create a new column with the full image path
df_sub_test['full_path_image'] = df_sub_test['image'].apply(join_paths)

<ipython-input-30-80e67b0f467c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub_test['full_path_image'] = df_sub_test['image'].apply(join_paths)


In [ ]:
df_sub_test.columns

Index(['questionId', 'question', 'image', 'docId', 'ucsf_document_id',
       'ucsf_document_page_no', 'data_split', 'full_path_image'],
      dtype='object')

In [ ]:
df_sub_test.head()

,questionId,question,image,docId,ucsf_document_id,ucsf_document_page_no,data_split,full_path_image
0,57344,What is the dividend payout in 2012?,documents/rnbx0223_193.png,4720,rnbx0223,193,test,/content/drive/MyDrive/docVqa_dataset/test/doc...
1,16384,What is the name of the person in the CC field ?,documents/lflm0081_1.png,5160,lflm0081,1,test,/content/drive/MyDrive/docVqa_dataset/test/doc...
2,57346,What is the % of Employees in 2012 based on gr...,documents/rnbx0223_191.png,4753,rnbx0223,191,test,/content/drive/MyDrive/docVqa_dataset/test/doc...
3,61870,What is the personnel costs in the 4th year?,documents/hrfw0227_24.png,8103,hrfw0227,24,test,/content/drive/MyDrive/docVqa_dataset/test/doc...
4,57348,What is the % of 'Providers of Capital' in the...,documents/rnbx0223_191.png,4753,rnbx0223,191,test,/content/drive/MyDrive/docVqa_dataset/test/doc...


In [ ]:
def extract_ocr_info(df, base_dir):

  dir_list = os.listdir(base_dir)
  ocr_base_dir = os.path.join(base_dir, dir_list[1])

  # Create empty lists for bounding boxes and word lists
  bbox_list = []
  word_list = []

  # Iterate over each row in the DataFrame
  for _, row in df.iterrows():

    # Construct the ocr_path for the row
    ocr_path = os.path.join(ocr_base_dir, row['ucsf_document_id'] + '_' + row['ucsf_document_page_no'] + '.json')

    # Open the json file and load the data
    with open(ocr_path, 'r') as f:
      ocr_json = json.load(f)

    # Extract the bounding boxes and word lists from the json data
    bbox_row = []
    word_row = []
    for i in ocr_json['recognitionResults'][0]['lines']:
      for j in i['words']:
        bbox_row.append(j['boundingBox'])
        word_row.append(j['text'])

    # Append the bbox and word lists to the global lists
    bbox_list.append(bbox_row)
    word_list.append(word_row)
  # Create new columns in the DataFrame to store the bbox and words lists
  df['bounding_boxes'] = bbox_list
  df['word_list'] = word_list

  df.drop(['questionId', 'image','ucsf_document_id', 'ucsf_document_page_no'], axis=1, inplace=True)

  # Return the updated DataFrame
  return df

In [ ]:
df_sub_test_ocr = extract_ocr_info(df_sub_test, base_test_path)

<ipython-input-33-9bb3011757be>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bounding_boxes'] = bbox_list
<ipython-input-33-9bb3011757be>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['word_list'] = word_list
<ipython-input-33-9bb3011757be>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['question

In [ ]:
df_sub_test_ocr.columns

Index(['question', 'docId', 'data_split', 'full_path_image', 'bounding_boxes',
       'word_list'],
      dtype='object')

In [ ]:
## Convert dataframe into dataset object
hf_sub_test_ocr = Dataset.from_pandas(df_sub_test_ocr)

In [ ]:
hf_sub_test_ocr

Dataset({
    features: ['question', 'docId', 'data_split', 'full_path_image', 'bounding_boxes', 'word_list'],
    num_rows: 10
})

In [ ]:
hf_sub_test_ocr.column_names

['question',
 'docId',
 'data_split',
 'full_path_image',
 'bounding_boxes',
 'word_list']

## Comparing all the column names from train, validation and test before proceding to next step:

In [ ]:
hf_sub_train_ocr.column_names   # fro training dataset

['question',
 'docId',
 'answers',
 'data_split',
 'full_path_image',
 'bounding_boxes',
 'word_list']

In [ ]:
hf_sub_valid_ocr.column_names   # for validation dataset

['question',
 'docId',
 'answers',
 'data_split',
 'full_path_image',
 'bounding_boxes',
 'word_list']

In [ ]:
hf_sub_test_ocr.column_names    # for test dataset

['question',
 'docId',
 'data_split',
 'full_path_image',
 'bounding_boxes',
 'word_list']

### ** Note from column names:
- test dataset doesn't contain answer column which is expected that our model will generate it

## Exp_5: Creating a single custom function for all type dataset i.e. train, validation and test

In [ ]:
def process_data(main_json_path, base_dir_path):

  # Open the JSON file
  with open(main_json_path, 'r') as file:
      # Load the contents of the file into a dictionary
      data = json.load(file)

  # Convert 'data' list into a dataframe
  df = pd.DataFrame(data['data'])

  # Define a function to join the base path with the image path
  def join_paths(image_path):
      return os.path.join(base_dir_path, image_path)

  # Apply the function to create a new column with the full image path
  df['full_path_image'] = df['image'].apply(join_paths)

  def extract_ocr_info(col, ocr_base_dir):

    # Construct the ocr_path for the col
    ocr_path = os.path.join(ocr_base_dir, col['ucsf_document_id'] + '_' + col['ucsf_document_page_no'] + '.json')

    # Open the json file and load the data
    with open(ocr_path, 'r') as f:
      ocr_json = json.load(f)

    # Extract the bounding boxes and word lists from the json data
    bbox_row = []
    word_row = []
    for i in ocr_json['recognitionResults'][0]['lines']:
      for j in i['words']:
        bbox_row.append(j['boundingBox'])
        word_row.append(j['text'])

    # Return the bounding boxes and word lists
    return bbox_row, word_row

  ocr_base_dir = os.path.join(base_dir_path, os.listdir(base_dir_path)[1])

  # Apply the extract_ocr_info function to each row in the DataFrame
  df['bounding_boxes'], df['word_list'] = zip(*df.apply(lambda col: extract_ocr_info(col, ocr_base_dir), axis=1))

  df.drop(['questionId', 'image', 'ucsf_document_id', 'ucsf_document_page_no'], axis=1, inplace=True)

  return df

In [ ]:
# training usage:
train_path_main = '/content/drive/MyDrive/docVqa_dataset/train/train_v1.0.json'
base_train_path = '/content/drive/MyDrive/docVqa_dataset/train'


In [ ]:
df_train_cf = process_data(train_path_main, base_train_path)

In [ ]:
df_train_cf.head()

,question,docId,answers,data_split,full_path_image,bounding_boxes,word_list
0,what is the date mentioned in this letter?,279,[1/8/93],train,/content/drive/MyDrive/docVqa_dataset/train/do...,"[[586, 30, 993, 36, 994, 119, 584, 131], [1080...","[Confidential, .., .., RJRT, PR, APPROVAL, DAT..."
1,what is the contact person name mentioned in l...,279,"[P. Carter, p. carter]",train,/content/drive/MyDrive/docVqa_dataset/train/do...,"[[586, 30, 993, 36, 994, 119, 584, 131], [1080...","[Confidential, .., .., RJRT, PR, APPROVAL, DAT..."
2,Which corporation's letterhead is this?,280,[Brown & Williamson Tobacco Corporation],train,/content/drive/MyDrive/docVqa_dataset/train/do...,"[[733, 196, 789, 193, 791, 218, 734, 219], [47...","[B&W, BROWN, &, WILLIAMSON, TOBACCO, CORPORATI..."
3,Who is in cc in this letter?,280,[T.F. Riehl],train,/content/drive/MyDrive/docVqa_dataset/train/do...,"[[733, 196, 789, 193, 791, 218, 734, 219], [47...","[B&W, BROWN, &, WILLIAMSON, TOBACCO, CORPORATI..."
4,what is the subject of this letter?,280,[Review of existing Brainstorming Ideas/483],train,/content/drive/MyDrive/docVqa_dataset/train/do...,"[[733, 196, 789, 193, 791, 218, 734, 219], [47...","[B&W, BROWN, &, WILLIAMSON, TOBACCO, CORPORATI..."


In [ ]:
df_train_cf.columns

Index(['question', 'docId', 'answers', 'data_split', 'full_path_image',
       'bounding_boxes', 'word_list'],
      dtype='object')

In [ ]:
# Convert dataframe into dataset object
hf_train_cf = Dataset.from_pandas(df_train_cf)

In [ ]:
hf_train_cf

Dataset({
    features: ['question', 'docId', 'answers', 'data_split', 'full_path_image', 'bounding_boxes', 'word_list'],
    num_rows: 39463
})

In [ ]:
## Checking whether our previously created huggingface training data column is same as this one or not
hf_sub_train_ocr.column_names == hf_train_cf.column_names   # yes that is true

True

In [ ]:
hf_train_cf[0]

{'question': 'what is the date mentioned in this letter?',
 'docId': 279,
 'answers': ['1/8/93'],
 'data_split': 'train',
 'full_path_image': '/content/drive/MyDrive/docVqa_dataset/train/documents/xnbl0037_1.png',
 'bounding_boxes': [[586, 30, 993, 36, 994, 119, 584, 131],
  [1080, 34, 1097, 32, 1099, 54, 1081, 55],
  [1102, 32, 1122, 31, 1124, 53, 1103, 54],
  [699, 203, 779, 204, 779, 234, 699, 234],
  [793, 204, 832, 204, 832, 234, 793, 234],
  [848, 204, 995, 204, 995, 237, 848, 234],
  [259, 296, 330, 295, 331, 322, 259, 323],
  [335, 295, 343, 295, 344, 322, 337, 322],
  [400, 258, 556, 261, 555, 332, 399, 329],
  [426, 334, 523, 331, 522, 410, 426, 412],
  [548, 331, 644, 333, 643, 413, 547, 411],
  [259, 416, 403, 414, 405, 445, 260, 444],
  [422, 413, 551, 412, 553, 447, 423, 445],
  [570, 411, 657, 410, 659, 449, 572, 447],
  [688, 400, 764, 398, 766, 457, 690, 464],
  [777, 398, 972, 393, 972, 445, 778, 456],
  [260, 474, 314, 473, 315, 505, 260, 505],
  [330, 473, 462, 472,

In [ ]:
# validation usage:
valid_path_main = '/content/drive/MyDrive/docVqa_dataset/val/val_v1.0.json'
base_valid_path = '/content/drive/MyDrive/docVqa_dataset/val'

In [ ]:
df_valid_cf = process_data(valid_path_main, base_valid_path)

In [ ]:
df_valid_cf.columns

Index(['question', 'docId', 'answers', 'data_split', 'full_path_image',
       'bounding_boxes', 'word_list'],
      dtype='object')

In [ ]:
# Convert dataframe into dataset object
hf_valid_cf = Dataset.from_pandas(df_valid_cf)

In [ ]:
hf_valid_cf.column_names

['question',
 'docId',
 'answers',
 'data_split',
 'full_path_image',
 'bounding_boxes',
 'word_list']

In [ ]:
hf_sub_valid_ocr.column_names == hf_valid_cf.column_names

True

In [ ]:
hf_valid_cf[0]

{'question': 'What is the ‘actual’ value per 1000, during the year 1975?',
 'docId': 14465,
 'answers': ['0.28'],
 'data_split': 'val',
 'full_path_image': '/content/drive/MyDrive/docVqa_dataset/val/documents/pybv0228_81.png',
 'bounding_boxes': [[1083, 226, 1209, 226, 1208, 259, 1082, 261],
  [1226, 226, 1261, 227, 1260, 262, 1225, 260],
  [1268, 227, 1301, 228, 1300, 265, 1267, 262],
  [645, 355, 717, 356, 718, 395, 645, 395],
  [738, 356, 926, 357, 927, 396, 738, 395],
  [944, 357, 1065, 357, 1066, 396, 945, 396],
  [1086, 357, 1251, 357, 1252, 395, 1087, 396],
  [1272, 357, 1463, 356, 1464, 395, 1273, 395],
  [1479, 356, 1693, 355, 1694, 393, 1480, 395],
  [1711, 355, 1806, 355, 1808, 393, 1712, 393],
  [422, 416, 492, 416, 492, 450, 422, 450],
  [1154, 392, 1293, 393, 1295, 427, 1155, 427],
  [401, 574, 492, 574, 492, 610, 400, 610],
  [401, 737, 492, 740, 491, 778, 400, 775],
  [401, 907, 492, 907, 491, 941, 401, 941],
  [404, 1068, 493, 1070, 492, 1107, 403, 1105],
  [421, 1233,

In [ ]:
# test usage:
test_path_main = '/content/drive/MyDrive/docVqa_dataset/test/test_v1.0.json'
base_test_path = '/content/drive/MyDrive/docVqa_dataset/test'

In [ ]:
df_test_cf = process_data(test_path_main, base_test_path)

In [ ]:
df_test_cf.columns

Index(['question', 'docId', 'data_split', 'full_path_image', 'bounding_boxes',
       'word_list'],
      dtype='object')

In [ ]:
# Convert dataframe into dataset object
hf_test_cf = Dataset.from_pandas(df_test_cf)

In [ ]:
hf_test_cf.column_names

['question',
 'docId',
 'data_split',
 'full_path_image',
 'bounding_boxes',
 'word_list']

In [ ]:
hf_sub_test_ocr.column_names == hf_test_cf.column_names

True

In [ ]:
hf_test_cf[0]

{'question': 'What is the dividend payout in 2012?',
 'docId': 4720,
 'data_split': 'test',
 'full_path_image': '/content/drive/MyDrive/docVqa_dataset/test/documents/rnbx0223_193.png',
 'bounding_boxes': [[1103, 52, 1147, 52, 1147, 83, 1103, 83],
  [1153, 53, 1247, 53, 1247, 84, 1153, 83],
  [1253, 53, 1265, 53, 1266, 84, 1253, 84],
  [1271, 53, 1357, 54, 1358, 84, 1272, 84],
  [1364, 54, 1414, 54, 1414, 84, 1364, 84],
  [1420, 54, 1538, 54, 1539, 84, 1420, 84],
  [1544, 54, 1610, 54, 1611, 83, 1545, 84],
  [882, 144, 932, 144, 931, 170, 882, 171],
  [937, 144, 1017, 146, 1016, 172, 937, 170],
  [723, 196, 791, 199, 790, 219, 722, 216],
  [1487, 206, 1499, 205, 1499, 229, 1486, 230],
  [1504, 205, 1569, 206, 1569, 230, 1503, 229],
  [108, 259, 169, 259, 169, 281, 109, 281],
  [179, 259, 225, 259, 225, 282, 179, 281],
  [882, 260, 945, 259, 945, 281, 883, 283],
  [952, 259, 999, 260, 1000, 282, 953, 281],
  [1523, 431, 1528, 370, 1547, 372, 1542, 432],
  [123, 719, 120, 677, 140, 676, 1